In [7]:
import pandas as pd

train = pd.read_csv("ViBidLQA/train.csv")
dev = pd.read_csv("ViBidLQA/dev.csv")
test = pd.read_csv("ViBidLQA/test.csv")

In [2]:
train.head()

,context,question,answer
0,Luật Đấu thầu số 22/2023/QH15 Điều 5...,Luật nào quy định về trường hợp cơ...,Luật Đấu thầu
1,Luật Đấu thầu số 22/2023/QH15 Điều 6...,Điều nào trong Luật Đấu thầu số 22/...,Điều 61
2,điểm a khoản 1 Điều 56 của Luật Đấu ...,Những cách tham vấn thị trường là gi...,các phương tiện thông tin phù hợp; ng...
3,Luật Đấu thầu số 22/2023/QH15 Điều 9...,Nhà thầu phải làm gìtrong thời hạn 10...,gửi đơn kiến nghị đến người có tha...
4,Nghị định hướng dẫn Luật Đấu thầu...,Điều kiện nào trước khi đơn vị thẩ...,tổ chức họp giữa các bên để trao đo...


In [3]:
train.shape

(1928, 3)

In [4]:
train['answer'][2]

'các phương tiện thông tin phù hợp; nghiên cứu catalô và các tài liệu giới thiệu sản phẩm của hãng sản xuất, nhà cung cấp, nhà phân phối, đại lý; tham khảo kinh nghiệm của các chủ đầu tư khác'

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
import os
from dotenv import load_dotenv
from legal_rag import LegalRAGSystem
from tqdm.auto import tqdm

# Tải biến môi trường (ví dụ: GOOGLE_API_KEY)
load_dotenv()

# Khởi tạo hệ thống RAG
# Đảm bảo rằng vectorstore của bạn đã được tạo từ trước
rag_system = LegalRAGSystem()
print("Hệ thống LegalRAG đã sẵn sàng.")

# Lấy một mẫu nhỏ của bộ test để chạy nhanh hơn (ví dụ: 20 mẫu)
# Bạn có thể tăng số lượng này để có kết quả đánh giá chính xác hơn
test_sample = test.head(20)

results = []
for index, row in tqdm(test_sample.iterrows(), total=test_sample.shape[0]):
    question = row['question']
    
    # Lấy kết quả từ RAG system
    generated_answer = rag_system.query(question)
    
    # Lấy các context đã được truy xuất
    # Chúng ta sẽ sử dụng hàm debug_chain_inputs đã tạo trước đó
    retrieved_contexts = rag_system.debug_chain_inputs(question)['context']
    
    results.append({
        "question": question,
        "generated_answer": generated_answer,
        "retrieved_contexts": [doc.page_content for doc in retrieved_contexts],
        "ground_truth": row['answer'] # Câu trả lời gốc từ dataset
    })

# Chuyển kết quả thành DataFrame để dễ xem
results_df = pd.DataFrame(results)
print("Generated results:")
print(results_df.head())


# Chuẩn bị dataset cho RAGAs
# Đổi tên cột để phù hợp với yêu cầu của RAGAs
ragas_dataset_dict = {
    'question': [r['question'] for r in results],
    'answer': [r['generated_answer'] for r in results],
    'contexts': [r['retrieved_contexts'] for r in results],
    'ground_truth': [r['ground_truth'] for r in results]
}
ragas_dataset = Dataset.from_dict(ragas_dataset_dict)

# Thực hiện đánh giá
# Lưu ý: Quá trình này sẽ gọi đến API của LLM (ví dụ: OpenAI hoặc Gemini) để chấm điểm
# nên có thể mất một chút thời gian và chi phí.
print("\nEvaluating with RAGAs...")
score = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
)

# In kết quả
print(score)
score_df = score.to_pandas()
score_df.head()

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


NameError: name 'results' is not defined

### 3. Đánh giá với RAGAs

Bây giờ, chúng ta sẽ định dạng lại dữ liệu và sử dụng RAGAs để chấm điểm. Các chỉ số chính bao gồm:
- **Faithfulness**: Câu trả lời có trung thực với context đã cho không?
- **Answer Relevancy**: Câu trả lời có liên quan đến câu hỏi không?
- **Context Precision & Recall**: Các context được truy xuất có chính xác và đầy đủ không?

In [ ]:
# Lấy một mẫu nhỏ của bộ test để chạy nhanh hơn (ví dụ: 20 mẫu)
# Bạn có thể tăng số lượng này để có kết quả đánh giá chính xác hơn
test_sample = test.head(20)

results = []
for index, row in tqdm(test_sample.iterrows(), total=test_sample.shape[0]):
    question = row['question']
    
    # Lấy kết quả từ RAG system
    generated_answer = rag_system.query(question)
    
    # Lấy các context đã được truy xuất
    # Chúng ta sẽ sử dụng hàm debug_chain_inputs đã tạo trước đó
    retrieved_contexts = rag_system.debug_chain_inputs(question)['context']
    
    results.append({
        "question": question,
        "generated_answer": generated_answer,
        "retrieved_contexts": [doc.page_content for doc in retrieved_contexts],
        "ground_truth": row['answer'] # Câu trả lời gốc từ dataset
    })

# Chuyển kết quả thành DataFrame để dễ xem
results_df = pd.DataFrame(results)
results_df.head()

### 2. Tạo câu trả lời từ RAG System

Chúng ta sẽ chạy các câu hỏi từ bộ dữ liệu `test` qua hệ thống RAG để thu thập các câu trả lời do AI tạo ra và các ngữ cảnh (context) mà nó đã sử dụng.

In [ ]:
import os
from dotenv import load_dotenv
from legal_rag import LegalRAGSystem
from tqdm.auto import tqdm

# Tải biến môi trường (ví dụ: GOOGLE_API_KEY)
load_dotenv()

# Khởi tạo hệ thống RAG
# Đảm bảo rằng vectorstore của bạn đã được tạo từ trước
rag_system = LegalRAGSystem()
print("Hệ thống LegalRAG đã sẵn sàng.")

### 1. Chuẩn bị môi trường và hệ thống RAG

Bây giờ, chúng ta sẽ import các thư viện cần thiết và khởi tạo `LegalRAGSystem` để sẵn sàng cho việc đánh giá.